In [85]:
#import semua library yang dibutuhkan
import csv
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [86]:
import pandas as pd
df = pd.read_csv('Dataset/BBC News Train.csv')
df

,ArticleId,Text,Category
0,1833,worldcom ex-boss launches defence lawyers defe...,business
1,154,german business confidence slides german busin...,business
2,1101,bbc poll indicates economic gloom citizens in ...,business
3,1976,lifestyle governs mobile choice faster bett...,tech
4,917,enron bosses in $168m payout eighteen former e...,business
...,...,...,...
1485,857,double eviction from big brother model caprice...,entertainment
1486,325,dj double act revamp chart show dj duo jk and ...,entertainment
1487,1590,weak dollar hits reuters revenues at media gro...,business
1488,1587,apple ipod family expands market apple has exp...,tech


In [87]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1490 entries, 0 to 1489
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ArticleId  1490 non-null   int64 
 1   Text       1490 non-null   object
 2   Category   1490 non-null   object
dtypes: int64(1), object(2)
memory usage: 35.0+ KB


In [88]:
#Memahami bentuk data
dataset = 'Dataset/BBC News Train.csv'
test_file = 'Dataset/BBC News Test.csv'
with open(dataset) as csvfile:
    print('Berikut adalah nama-nama kolom:\n\n', csvfile.readline())
    print('konten file adalah sebagai berikut:\n\n', csvfile.readline())

Berikut adalah nama-nama kolom:

 ArticleId,Text,Category

konten file adalah sebagai berikut:

 1833,worldcom ex-boss launches defence lawyers defending former worldcom chief bernie ebbers against a battery of fraud charges have called a company whistleblower as their first witness.  cynthia cooper  worldcom s ex-head of internal accounting  alerted directors to irregular accounting practices at the us telecoms giant in 2002. her warnings led to the collapse of the firm following the discovery of an $11bn (Â£5.7bn) accounting fraud. mr ebbers has pleaded not guilty to charges of fraud and conspiracy.  prosecution lawyers have argued that mr ebbers orchestrated a series of accounting tricks at worldcom  ordering employees to hide expenses and inflate revenues to meet wall street earnings estimates. but ms cooper  who now runs her own consulting business  told a jury in new york on wednesday that external auditors arthur andersen had approved worldcom s accounting in early 2001 and 2002

In [89]:
#Buat fungsi untuk membuang semua stopping words, yaitu kata2 yang tidak memiliki makna berarti (general)
def stopwords_remove(sentence):
    stopwords = ["s", "mr", "ms", "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]
    sentences = []
    for word in sentence.lower().split():
        if word.strip('.') in stopwords:
            pass
        else:
            sentences.append(word.strip('.'))
    sentences = ' '.join(sentences)
    return sentences   
        

In [90]:
#Buang semua stopping words, yaitu kata2 yang tidak memiliki makna berarti (general)
#dan ambil kolom berita('Text'), dan kategorinya ('Category')
def parsing_file(file):
    sentences = []
    labels = []
    
    with open(file) as csvfile:
        next(csvfile)
        csvfile = csv.reader(csvfile, delimiter=',')
        for row in csvfile:
            row[1] = stopwords_remove(row[1])
            sentences.append(row[1])
            labels.append(row[2])
        
    return sentences, labels

In [91]:
dataset_sentences, dataset_labels = parsing_file(dataset)
len(dataset_sentences)

1490

In [92]:
#Preprocess the test data
test_id = []
test_sentences = []

with open(test_file) as csvfile:
        next(csvfile)
        csvfile = csv.reader(csvfile, delimiter=',')
        for row in csvfile:
            row[1] = stopwords_remove(row[1])
            test_id.append(row[0])
            test_sentences.append(row[1])

In [93]:
len(test_sentences)
len(test_id)

735

In [94]:
# len(train_sentences[1].split())
# train_labels[:5]
# len(train_sentences)

## Tokenizer

In [95]:
#parameter
oov_token = '<OOV>'
vocab_size = 10000

#Build function for tokenize sentences
def fit_tokenizer(sentences):
    #inisiate the tokenizer class
    tokenizer = Tokenizer(oov_token=oov_token, num_words=vocab_size)
    #fit tokenizer to text
    tokenizer.fit_on_texts(sentences)
    
    return tokenizer

In [96]:
dataset_token = fit_tokenizer(dataset_sentences)

In [97]:
dataset_word_index = dataset_token.word_index

In [98]:
# len(train_word_index)

## Splitting data

In [115]:
#data splitting
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(dataset_sentences,
                                                    dataset_labels,
                                                    test_size=0.3,
                                                    stratify = dataset_labels,
                                                    random_state=0)

In [100]:
#parameter
maxlen = 1500
truncating = 'post'

def generate_pad_seq(tokenizer, sentences):    
    #algoritma
    sequences = tokenizer.texts_to_sequences(sentences)
    padded = pad_sequences(sequences, maxlen=maxlen, truncating=truncating)
    
    return padded

In [118]:
X_train = generate_pad_seq(dataset_token, X_train)
# X_test = generate_pad_seq(dataset_token, X_test)
 
test_sequences = dataset_token.texts_to_sequences(X_test)
test_padded = pad_sequences(test_sequences, maxlen=maxlen)
X_test = test_padded

In [119]:
type(train_labels)

numpy.ndarray

In [120]:
X_train

array([[   0,    0,    0, ..., 1107,  802, 4067],
       [   0,    0,    0, ..., 2456,    1,  701],
       [   0,    0,    0, ..., 1422,  313, 1729],
       ...,
       [   0,    0,    0, ..., 1361,   68,  779],
       [   0,    0,    0, ...,  708,  310,  160],
       [   0,    0,    0, ..., 2450, 8036,    1]])

In [103]:
len(X_train) + len(X_test)

1490

In [121]:
#Encode the category label
from sklearn.preprocessing import LabelEncoder
import numpy as np
import tensorflow as tf

le = LabelEncoder()

train_labels = le.fit_transform(y_train)
train_labels = np.array(tf.keras.utils.to_categorical(train_labels))
test_labels = le.fit_transform(y_test)
test_labels = np.array(tf.keras.utils.to_categorical(test_labels))

In [123]:
train_labels.shape

(1043, 5)

In [105]:
#Build tensorflow dataset from train and test data
train_ds = tf.data.Dataset.from_tensor_slices((X_train,train_labels))
test_ds = tf.data.Dataset.from_tensor_slices((X_test,test_labels))

In [106]:
#Mengecek bagaimana bentuk datanya
# count =0
# print('======Train dataset ====')
# for value,label in train_ds:
#     count += 1
#     print('Sentence:', value, '\nLabel:', label)
#     if count==3:
#         break
# count =0

======Train dataset ====
Sentence: tf.Tensor([   0    0    0 ... 1107  802 4067], shape=(1500,), dtype=int32) 
Label: tf.Tensor([0. 0. 0. 1. 0.], shape=(5,), dtype=float32)
Sentence: tf.Tensor([   0    0    0 ... 2456    1  701], shape=(1500,), dtype=int32) 
Label: tf.Tensor([1. 0. 0. 0. 0.], shape=(5,), dtype=float32)
Sentence: tf.Tensor([   0    0    0 ... 1422  313 1729], shape=(1500,), dtype=int32) 
Label: tf.Tensor([0. 0. 1. 0. 0.], shape=(5,), dtype=float32)


## Building Model

### Plot utility
Define the function to visualize the accuracy

In [107]:
import matplotlib.pyplot as plt

# Plot Utility
def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()

### Defining the model

In [126]:
# Parameters
embedding_dim = 16
lstm_dim = 32
dense_dim = 6

# Model Definition with LSTM
model_lstm = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=maxlen),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_dim)),
    tf.keras.layers.Dense(dense_dim, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])

# Set the training parameters
model_lstm.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

# model summary
model_lstm.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 1500, 16)          160000    
                                                                 
 bidirectional_3 (Bidirectio  (None, 64)               12544     
 nal)                                                            
                                                                 
 dense_6 (Dense)             (None, 6)                 390       
                                                                 
 dense_7 (Dense)             (None, 5)                 35        
                                                                 
Total params: 172,969
Trainable params: 172,969
Non-trainable params: 0
_________________________________________________________________


In [127]:
NUM_EPOCHS = 10
BATCH_SIZE = 128

# Train the model
history_lstm = model_lstm.fit(X_train, train_labels, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS, validation_data=(y_test, test_labels))

Epoch 1/10
9/9 [==============================] - ETA: 0s - loss: 1.6083 - accuracy: 0.2253

ValueError: Failed to find data adapter that can handle input: (<class 'list'> containing values of types {"<class 'str'>"}), <class 'numpy.ndarray'>

In [129]:
from tensorflow.keras import regularizers
# max_features =20000
embedding_dim =64
# sequence_length = 40

model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=maxlen,
                                    embeddings_regularizer = regularizers.l2(0.0005)))                                    

model.add(tf.keras.layers.Conv1D(128,3, activation='relu',
                                 kernel_regularizer = regularizers.l2(0.0005),
                                 bias_regularizer = regularizers.l2(0.0005)))                               
model.add(tf.keras.layers.GlobalMaxPooling1D())
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(5, activation='sigmoid',
                                kernel_regularizer=regularizers.l2(0.001),
                                bias_regularizer=regularizers.l2(0.001),))
                               



model.summary()
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), optimizer='Nadam', metrics=["CategoricalAccuracy"])

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 1500, 64)          640000    
                                                                 
 conv1d (Conv1D)             (None, 1498, 128)         24704     
                                                                 
 global_max_pooling1d (Globa  (None, 128)              0         
 lMaxPooling1D)                                                  
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_8 (Dense)             (None, 5)                 645       
                                                                 
Total params: 665,349
Trainable params: 665,349
Non-trainable params: 0
________________________________________________

In [130]:
epochs = 100
# Fit the model using the train and test datasets.
#history = model.fit(x_train, train_labels,validation_data= (x_test,test_labels),epochs=epochs )
history = model.fit(train_ds.shuffle(2000).batch(128),
                    epochs= epochs ,
                    validation_data=test_ds.batch(128),
                    verbose=1)

Epoch 1/100


C:\Users\asus\anaconda3\lib\site-packages\tensorflow\python\util\dispatch.py:1082: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


9/9 [==============================] - 20s 2s/step - loss: 1.8992 - categorical_accuracy: 0.2339 - val_loss: 1.8552 - val_categorical_accuracy: 0.3289
Epoch 2/100
9/9 [==============================] - 6s 626ms/step - loss: 1.8156 - categorical_accuracy: 0.3327 - val_loss: 1.7972 - val_categorical_accuracy: 0.4385
Epoch 3/100
9/9 [==============================] - 6s 616ms/step - loss: 1.7467 - categorical_accuracy: 0.4314 - val_loss: 1.7417 - val_categorical_accuracy: 0.4609
Epoch 4/100
9/9 [==============================] - 6s 627ms/step - loss: 1.6776 - categorical_accuracy: 0.5120 - val_loss: 1.6832 - val_categorical_accuracy: 0.5168
Epoch 5/100
9/9 [==============================] - 6s 613ms/step - loss: 1.6074 - categorical_accuracy: 0.5983 - val_loss: 1.6125 - val_categorical_accuracy: 0.6219
Epoch 6/100
9/9 [==============================] - 5s 604ms/step - loss: 1.5122 - categorical_accuracy: 0.7287 - val_loss: 1.5209 - val_categorical_accuracy: 0.7360
Epoch 7/100
9/9 [=======

9/9 [==============================] - 6s 615ms/step - loss: 0.2023 - categorical_accuracy: 0.9952 - val_loss: 0.3605 - val_categorical_accuracy: 0.9441
Epoch 51/100
9/9 [==============================] - 6s 620ms/step - loss: 0.2025 - categorical_accuracy: 0.9971 - val_loss: 0.3599 - val_categorical_accuracy: 0.9485
Epoch 52/100
9/9 [==============================] - 6s 624ms/step - loss: 0.1981 - categorical_accuracy: 0.9971 - val_loss: 0.3583 - val_categorical_accuracy: 0.9463
Epoch 53/100
9/9 [==============================] - 6s 631ms/step - loss: 0.2006 - categorical_accuracy: 0.9942 - val_loss: 0.3576 - val_categorical_accuracy: 0.9441
Epoch 54/100
9/9 [==============================] - 6s 653ms/step - loss: 0.1967 - categorical_accuracy: 0.9971 - val_loss: 0.3577 - val_categorical_accuracy: 0.9485
Epoch 55/100
9/9 [==============================] - 6s 624ms/step - loss: 0.1947 - categorical_accuracy: 0.9962 - val_loss: 0.3597 - val_categorical_accuracy: 0.9441
Epoch 56/100
9/9 

9/9 [==============================] - 6s 643ms/step - loss: 0.1579 - categorical_accuracy: 0.9990 - val_loss: 0.3476 - val_categorical_accuracy: 0.9351
Epoch 100/100
9/9 [==============================] - 6s 626ms/step - loss: 0.1562 - categorical_accuracy: 0.9990 - val_loss: 0.3475 - val_categorical_accuracy: 0.9351


In [ ]:
# train_sequences, train_padded = generate_pad_seq(train_token, train_sentences)

In [ ]:
# test_sequences, tes_padded = generate_pad_seq(train_token, test_sentences)

In [ ]:
# train_sequences[0]
# len(train_sequences[0])
# len(train_sentences[0].split())
# len(train_padded[0])
